# 0. Imports

In [ ]:
import os
import warnings

warnings.filterwarnings('ignore')

In [ ]:
# Manejo de datos
import numpy as np
import geopandas as gpd

# Visualización de datos
import pydeck as pdk
from keplergl import KeplerGl

# Utilidades
from gcsfs import GCSFileSystem

from research.utils.geocoder import Geocoder
from research.viz.plot import (
    category_color_intervals, 
    equal_color_intervals,
)

# 1. Leemos nuestros datasets

In [ ]:
gcs_bucket = os.getenv("GCS_BUCKET_URI")
fs = GCSFileSystem()

### 1.1 Carriles bici

Fuente: Portal de datos abiertos del Ayuntamiento de Madrid (2022) ([link](https://datos.madrid.es/sites/v/index.jsp?vgnextoid=325e827b864f4410VgnVCM2000000c205a0aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD))

In [ ]:
# Leemos nuestros datos de carriles bicicleta (vectorial, líneas)

carriles_gdf = gpd.read_file(f'{gcs_bucket}/carriles_bicicleta.gpkg')

carriles_gdf.head(3)

### 1.2 Accidentes bicicleta

Fuente: Accidentes de tráfico con implicación de bicicletas del portal de datos abiertos del Ayuntamiento de Madrid (2022) ([link](https://datos.madrid.es/portal/site/egob/menuitem.c05c1f754a33a9fbe4b2e4b284f1a5a0/?vgnextoid=20f4a87ebb65b510VgnVCM1000001d4a900aRCRD&vgnextchannel=374512b9ace9f310VgnVCM100000171f5a0aRCRD&vgnextfmt=default))

In [ ]:
# Leemos nuestro dataset de accidentes de bicicleta

accidentes_bicicletas_uri = f"{gcs_bucket}/accidentes_bicicleta_madrid_2022.csv"

accidentes_gdf = gpd.read_file(accidentes_bicicletas_uri)

accidentes_gdf.head()

# 2. Preparación de los datos: geocodificación

Para este caso de uso, vamos a hacer uso de las direcciones disponibles en nuestro dataset de accidentes para obtener a partir de éstas las geometrías correspondientes.

>**NOTA:** La fuente de origen si tiene coordenadas X e Y en UTM con lo que este paso no es necesario, pero se ha optado por llevarlo a cabo a modo demostrativo.

Para la geocodificación de los datos usaremos las utilidades de GeoPandas que a su vez hacen uso de la librería [GeoPy](https://geopy.readthedocs.io/en/stable/), compatible con una serie de proveedores distintos.

<img src="https://storage.googleapis.com/mmoncadaisla-public-access-data/images/geocoding.png" alt="geocoding" width="600"/>

In [ ]:
geocoder = Geocoder(provider="Photon") # Use 'Photon' for no API key usage and quota consumption

In [ ]:
# Descomentar la siguiente línea para correr la operación sobre todo el dataset

#accidentes_gdf['geometry'] = accidentes_gdf['direccion'].apply(geocoder.geocode_address) 

# Alternativamente, geocodificamos una muestra a modo demostrativo y leemos el dataset, ya geocodificado, desde GCS

gdf_subset = accidentes_gdf[:10].copy()

gdf_subset['geometry'] = gdf_subset['direccion'].apply(geocoder.geocode_address)

gdf_subset.head(3)

In [ ]:
# Para el resto del ejercicio usaremos el dataset ya procesado, leyendo la fuente del dato en formato (Geo)Parquet

accidentes_gdf = gpd.read_parquet(
    f'{gcs_bucket}/accidentes_bicicleta_madrid_2022_geocoded.parquet', 
    filesystem=fs
)

accidentes_gdf.head(3)

# 2. Visualización de datos geoespaciales

[Kepler.GL](https://docs.kepler.gl/) es un proyecto que emplea Deck.gl y Mapbox GL para la exploración visual de datos geoespaciales. La integración con Jupyter Notebook, aunque relativamente limitada, nos permite directamente cargar GeoDataFrames de forma fácil e intuitiva, lo cuál puede ser muy beneficioso para una exploración inicial del dato.

## A) Con Kepler GL

In [ ]:
esda_map = KeplerGl(height=600)

esda_map.add_data(accidentes_gdf, name='Accidentes bicicleta')
esda_map.add_data(carriles_gdf, name='Carriles bici')

In [ ]:
esda_map

## B) Mediante PyDeck 

[PyDeck](https://pydeck.gl/) es un proyecto que crear visualizaciones de datos geoespaciales de distinto tipo mediante [Deck.gl](https://deck.gl/#/), directamente desde nuestro entorno de Python

### Creamos una visualización sencilla

In [ ]:
# Define the layer to display in Pydeck use expression to style by radius

layer = pdk.Layer('GeoJsonLayer', # E.g: switch to 'HeatmapLayer'
                  data=accidentes_gdf,
                  get_position='geometry.coordinates',
                  get_point_radius='30 + (cod_lesividad * 2)',
                  get_fill_color=[178, 34, 34],
                  radius_pixels=20,
                  pickable=True,
        )

lanes_layer = pdk.Layer('GeoJsonLayer',
                         data=carriles_gdf,
                         get_line_color=[26, 207, 196],
                         get_line_width=5,
                       )

# Define the view state and set the initial camera position
view_state = pdk.ViewState(latitude=40.4168, longitude=-3.7038, zoom=13)

# Define the map object and display the layer
viz = pdk.Deck(
    layers=[lanes_layer, layer], 
    initial_view_state=view_state, 
    map_provider='carto', # E.g: Switch to 'google_maps'
    map_style='light', # E.g: Switch to 'satellite'
)

viz.to_html()

### Vamos a ayudar 'style helpers' para estilar los accidentes por severidad

In [ ]:
# Define the layer to display in Pydeck use expression to style by radius & add style helper

style_expression, legend = equal_color_intervals(
    accidentes_gdf, 
    'cod_lesividad', 
    'sunset', 
    5, 
    return_legend=True,
    legend_title='Lesividad del accidente',
)


display(legend)

layer = pdk.Layer('GeoJsonLayer',
                  data=accidentes_gdf,
                  get_position='geometry.coordinates',
                  get_point_radius=20,
                  get_fill_color=style_expression,
                  pickable=True,
                  auto_highlight=True,
        )


# Define the view state and set the initial camera position
view_state = pdk.ViewState(latitude=40.4168, longitude=-3.7038, zoom=13)

# Define the map object and display the layer
viz = pdk.Deck(
    layers=[lanes_layer, layer], 
    initial_view_state=view_state, 
    map_provider='carto',
    map_style='dark',
)

viz.to_html()

### Vamos a diferenciar también por tipo de vía en función de categoría

In [ ]:
# Define the layer to display in Pydeck use expression to style by radius & add style helper

vias_gdf = carriles_gdf[carriles_gdf['TIPO_VIA'].notna()]

lanes_style, lanes_legend = category_color_intervals(
    vias_gdf, 
    'TIPO_VIA', 
    'pastel', 
    return_legend=True, 
    legend_title='Tipo de vía'
)

display(lanes_legend)

lanes_layer = pdk.Layer('GeoJsonLayer',
                         data=vias_gdf,
                         get_line_color=lanes_style,
                         get_line_width=5,
                         pickable=True,
                         auto_highlight=True,
                       )

# Define the view state and set the initial camera position
view_state = pdk.ViewState(latitude=40.4168, longitude=-3.7038, zoom=14)

# Define the map object and display the layer
viz = pdk.Deck(
    layers=[lanes_layer, layer], 
    initial_view_state=view_state, 
    map_provider='carto',
    map_style='dark',
)

viz.to_html()

# Análisis de elementos vecinos con GeoPandas

Para encontrar el segmento de nuestra capa de carriles más cercano a cada accidente, podemos hacer uso del método [sjoin_nearest](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin_nearest.html) de GeoPandas. 

La distancia se calcula en función de las coordenadas, por lo que pasamos a pseudo-mercator ([EPSG:3857](https://epsg.io/3857)) para que las unidades sean metros.

>NOTA: Cálculos más precisos requerirían de una elección más ajustada del sistema de coordenadas dependiendo de la ubicación geográfica. Para nuestros datos podríamos haber empleado [EPSG:3042](https://epsg.io/3042).

<img src="https://storage.googleapis.com/mmoncadaisla-public-access-data/images/sjoin_nearest.png" alt="geocoding" width="400"/>

In [ ]:
# Aplicamos el método sjoin_nearest

accidentes_carril_gdf = accidentes_gdf.to_crs(epsg=3857).sjoin_nearest(
    carriles_gdf.to_crs(epsg=3857), 
    how='left', 
    distance_col='distancia'
)

accidentes_carril_gdf.head()

In [ ]:
# Quitamos la asignación de carril para todos aquellos elementos cuya distancia al carril más próximo supera 25m

threshold = 25 # 25 metros

accidentes_carril_gdf.loc[accidentes_carril_gdf['distancia'] > threshold, 'tipo_via_cercana'] = 'Ninguna'
accidentes_carril_gdf.loc[accidentes_carril_gdf['distancia'] > threshold, 'id_segmento'] = np.nan

accidentes_fuera_carril_gdf = accidentes_carril_gdf[
    accidentes_carril_gdf['tipo_via_cercana'] == 'Ninguna'][
    ['geometry', 'cod_lesividad', 'estado_meteorológico', 'tipo_accidente']
    ].copy()

accidentes_fuera_carril_gdf = accidentes_fuera_carril_gdf.to_crs(epsg=4326)

accidentes_carril_gdf.head(3)

In [ ]:
# Group using the bike lane ID to identify number of accidents per lane

accidentes_carriles = accidentes_carril_gdf.groupby(by='id_segmento')[['distancia']].count().rename(
                        columns={'distancia': 'num_accidentes'}
                        ).reset_index()


# Find the top 5 bike lanes with the highest number of accidents

accidentes_carriles.sort_values(by='num_accidentes', ascending=False).head(5)

In [ ]:
# Join the information with our bike lane GeoDataFrame and visualize on a map

accidentes_carriles_gdf = carriles_gdf.merge(
                            accidentes_carriles['num_accidentes'], 
                            left_on='id_segmento', 
                            right_on=accidentes_carriles['id_segmento'], 
                            how='left')

accidentes_carriles_gdf.sort_values(by='num_accidentes', ascending=False)

In [ ]:
# Define the layer to display in Pydeck use expression to style by radius & add style helper


accidents_out = pdk.Layer('GeoJsonLayer',
                  data=accidentes_fuera_carril_gdf,
                  get_position='geometry.coordinates',
                  get_point_radius='10 + (cod_lesividad * 2)',
                  get_fill_color=[134, 19, 218],
                  opacity=0.7,
                  pickable=True,
        )

lanes_style, lanes_legend = equal_color_intervals(
    accidentes_carriles_gdf, 
    'num_accidentes', 
    'TealRose',
    6,
    return_legend=True, 
    legend_title='Número de accidentes',
)

display(lanes_legend)

lanes_layer = pdk.Layer('GeoJsonLayer',
                         data=accidentes_carriles_gdf,
                         get_line_color=lanes_style,
                         get_line_width='15 + (0.5 * num_accidentes)',
                         pickable=True,
                         auto_highlight=True,
                       )

# Define the view state and set the initial camera position
view_state = pdk.ViewState(latitude=40.4168, longitude=-3.7038, zoom=14)

# Define the map object and display the layer
viz = pdk.Deck(
    layers=[lanes_layer, accidents_out], 
    initial_view_state=view_state, 
    map_provider='google_maps',
    map_style='satellite',
)

viz.to_html()

## TIP: Normalizar los datos con información sobre cantidad de ciclistas total en el área